<a href="https://colab.research.google.com/github/ciao-baby/roy_amato/blob/main/roy_amato_revised_deliverable_3_project_1_university_id_u01862158.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A cleaned, production-ready credibility Python module (sync, robust, testable).

A hybrid NN architecture (Transformer text encoder + metadata MLP) with PyTorch training skeleton and defense of assumptions (novelty).

Example code to package & push the model artifact to Hugging Face Hub.

A FastAPI integration endpoint suitable for a chatbot (scalable, simple).

Unit test examples and basic monitoring/CI suggestions.

In [ ]:
# credibility.py
# ============================================
# Production-ready URL credibility scoring module
# - Robust HTTP handling
# - Extracts metadata, content features and domain signals
# - Returns structured JSON suitable for a chatbot or API response
# ============================================

import time
import requests
from requests.adapters import HTTPAdapter, Retry
from bs4 import BeautifulSoup
import tldextract
from urllib.parse import urlparse
import hashlib

# --- HTTP session with retries ---
# Create a single Requests Session configured with retries and timeouts.
# Using session + Retry reduces transient network failure impact and improves reliability.
_session = requests.Session()
_retries = Retry(total=2, backoff_factor=0.5, status_forcelist=[429, 500, 502, 503, 504])
_session.mount("https://", HTTPAdapter(max_retries=_retries))
_session.mount("http://", HTTPAdapter(max_retries=_retries))

def _safe_get(url, timeout=6):
    """Fetch URL content safely with retries and return text and status code.
    - returns (text, status_code)
    - raises exception on fatal errors to be handled by callers
    """
    resp = _session.get(url, timeout=timeout, headers={"User-Agent": "CredBot/1.0"})
    resp.encoding = resp.apparent_encoding or resp.encoding
    return resp.text, resp.status_code

# --- Helper: canonicalize domain and scheme ---
# Extracts domain info and canonical host; used for domain-based heuristics and caching keys.
def _domain_info(url):
    parsed = tldextract.extract(url)
    domain = f"{parsed.domain}.{parsed.suffix}" if parsed.suffix else parsed.domain
    scheme = urlparse(url).scheme or "http"
    return domain.lower(), scheme.lower()

# --- Feature extraction from HTML/text ---
# Extract title, visible text, word counts, meta tags, and presence of login/news/ad patterns.
def _extract_features(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    title = (soup.title.string or "").strip() if soup.title else ""
    # visible text with separators, used for word-count and content-signal heuristics
    text = soup.get_text(separator=" ", strip=True)
    word_count = len(text.split())
    # meta tags helpful for credibility (author, published-time, canonical)
    meta = {}
    for name in ("author", "publisher", "article:published_time", "og:title", "og:site_name"):
        tag = soup.find("meta", attrs={"name": name}) or soup.find("meta", attrs={"property": name})
        if tag and tag.get("content"):
            meta[name] = tag["content"]
    # simple signals
    has_author = bool(meta.get("author"))
    has_pubtime = bool(meta.get("article:published_time"))
    contains_paywall = bool(soup.select_one("div.paywall") or "subscribe" in text.lower() and "read more" in text.lower())
    return {
        "title": title[:300],
        "text": text,
        "word_count": word_count,
        "meta": meta,
        "has_author": has_author,
        "has_pubtime": has_pubtime,
        "contains_paywall": contains_paywall
    }

# --- Core heuristic scorer ---
# Small, interpretable scoring function that uses domain, scheme and content heuristics.
def heuristic_score(domain, scheme, features):
    """
    Start from base and apply heuristic boosts/penalties:
    - HTTPS gives +10
    - .edu/.gov +20, established .org/.com +5
    - low word_count penalty, presence of author/published time bonus
    - suspicious TLDs penalized
    """
    score = 50
    if scheme == "https":
        score += 10

    if domain.endswith((".edu", ".gov")):
        score += 20
    elif domain.endswith((".org", ".com")):
        score += 5
    elif domain.endswith((".biz", ".info", ".xyz", ".top", ".pw")):
        score -= 15

    wc = features["word_count"]
    if wc < 200:
        score -= 12
    elif wc > 1000:
        score += 6

    # Author and published time as trust signals
    if features["has_author"]:
        score += 5
    if features["has_pubtime"]:
        score += 7
    if features["contains_paywall"]:
        score += 2  # paywalls often imply publisher business model (weak signal)

    # clamp
    score = max(0, min(100, score))
    return score

def score_url(url, fetch_timeout=6):
    """Top-level function returning structured result for a single URL.
    - returns dict with metadata, numeric score and label ("Low"/"Medium"/"High")
    - resilient: catches and reports exceptions without raising
    """
    start = time.time()
    try:
        html, status = _safe_get(url, timeout=fetch_timeout)
        domain, scheme = _domain_info(url)
        features = _extract_features(html)
        score = heuristic_score(domain, scheme, features)
        label = "Low" if score < 40 else ("Medium" if score < 70 else "High")
        uid = hashlib.sha256(url.encode("utf-8")).hexdigest()[:12]
        return {
            "url": url,
            "uid": uid,
            "domain": domain,
            "scheme": scheme,
            "status_code": status,
            "title": features["title"],
            "word_count": features["word_count"],
            "has_author": features["has_author"],
            "has_pubtime": features["has_pubtime"],
            "score": score,
            "label": label,
            "latency_ms": int((time.time() - start) * 1000)
        }
    except Exception as exc:
        return {
            "url": url,
            "error": str(exc),
            "score": 0,
            "label": "Unavailable",
            "latency_ms": int((time.time() - start) * 1000)
        }

# --- Batch helper for multiple URLs (synchronous) ---
# Simple convenience wrapper that processes list of URLs and returns results list.
def score_urls(urls, timeout=6):
    results = []
    for url in urls:
        results.append(score_url(url, fetch_timeout=timeout))
    return results

# If run as script: quick CLI
if __name__ == "__main__":
    print("🔍 Credibility scoring (production-ready toy). Enter URLs separated by spaces:")
    urls = input("> ").split()
    res = score_urls(urls)
    for r in res:
        if r.get("error"):
            print(f"URL: {r['url']}\n  ❌ {r['error']}\n")
        else:
            print(f"URL: {r['url']}\n  Domain: {r['domain']}\n  Score: {r['score']} ({r['label']})\n")


Problem: Credibility is multifaceted: content semantics, writing style, topicality (publication date), and domain-level trust.

Idea (novelty): Use a hybrid model: a pretrained Transformer (e.g., DistilBERT) encodes the article text into a semantic embedding; metadata features (domain-encoded embeddings, TLD, HTTPS flag, word count, author presence, published time delta, social signals if available) are concatenated and passed through an MLP. The combined network learns late fusion weights to judge credibility.

Why novel: The novelty is in explicitly learning attention-like gating between semantic and metadata channels: a small gating MLP outputs per-channel weights that adaptively weight semantic vs. metadata contributions for each sample. This counters the class assumption that "text-only" is sufficient — the model learns when to trust metadata more (e.g., short content from unknown TLDs) vs. semantic content (opinionated long-form text).

Assumptions & counterargument: Classic assumption: BERT-style semantics alone capture credibility. Counter: metadata (domain, author, timestamp) carry orthogonal signals. The gating mechanism lets the model adapt if metadata is noisy (learned weighting may downweight it). This trade-off and gating is the model novelty.

In [ ]:
# model.py
# 3-5 lines of comments per block describing purpose and functionality.
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class HybridCredModel(nn.Module):
    """
    Hybrid model:
    - Text encoder: pretrained Transformer (frozen or fine-tuned)
    - Metadata MLP to encode numeric/categorical features
    - Gating mechanism to weight channels and final MLP for regression/classification
    """
    def __init__(self, transformer_name="distilbert-base-uncased", meta_dim=16, freeze_transformer=False):
        super().__init__()
        # Text encoder: pretrained transformer -> pooled vector
        self.transformer = AutoModel.from_pretrained(transformer_name)
        if freeze_transformer:
            for p in self.transformer.parameters():
                p.requires_grad = False
        hidden_size = self.transformer.config.hidden_size

        # Metadata MLP: encodes numeric/categorical metadata into meta_dim vector
        self.meta_net = nn.Sequential(
            nn.Linear(10, 64),  # assume 10 numeric/categorical features preprocessed
            nn.ReLU(),
            nn.Linear(64, meta_dim),
            nn.ReLU()
        )

        # Gating network: produces two weights to combine text/meta
        self.gate = nn.Sequential(
            nn.Linear(hidden_size + meta_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.Softmax(dim=-1)
        )

        # Final predictor MLP -> scalar score (regression) or logits
        self.pred = nn.Sequential(
            nn.Linear(hidden_size + meta_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)  # regression score 0-100; use BCE or softmax for classification
        )

    def forward(self, input_ids, attention_mask, meta_features):
        # Text embedding: CLS/pool
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        # use pooled output (for BERT-like) or mean pool for some models
        if hasattr(out, "pooler_output") and out.pooler_output is not None:
            txt_emb = out.pooler_output
        else:
            # mean pool last_hidden_state
            txt_emb = (out.last_hidden_state * attention_mask.unsqueeze(-1)).sum(1)
            txt_emb = txt_emb / attention_mask.sum(1, keepdim=True)

        meta_emb = self.meta_net(meta_features)
        combined = torch.cat([txt_emb, meta_emb], dim=1)
        gates = self.gate(combined)  # [batch, 2]
        # weighted combination (not strictly necessary but demonstrates gating)
        gated = gates[:, 0:1] * txt_emb + gates[:, 1:2] * meta_emb
        final_in = torch.cat([txt_emb, meta_emb], dim=1)
        out_score = self.pred(final_in).squeeze(-1)
        return out_score, gates


Use MSE for regression to a 0–100 score or cross-entropy if mapping to classes.

Preprocess metadata so all features are numeric and scaled (one-hot for categorical TLD or domain clusters).

If labeled dataset is small: freeze transformer and train classifier + meta MLP; then optionally unfreeze last transformer layers.

Use mix of loss terms (primary MSE + optional regularization that penalizes too-large gate entropy to prevent collapse).

In [ ]:
# train.py (skeleton)
# - Load dataset of {url,text,meta_features,score}
# - Tokenize text with a transformer tokenizer and train HybridCredModel
import torch, random
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from model import HybridCredModel

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

class CredDataset(Dataset):
    def __init__(self, records):
        self.records = records
    def __len__(self): return len(self.records)
    def __getitem__(self, idx):
        rec = self.records[idx]
        toks = tokenizer(rec["text"][:2000], truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        meta = torch.tensor(rec["meta_features"], dtype=torch.float32)
        score = torch.tensor(rec["score"], dtype=torch.float32)
        return toks["input_ids"].squeeze(0), toks["attention_mask"].squeeze(0), meta, score

# Example: prepare records list with 'text','meta_features'(len=10),'score'
train_records = []  # populate with real data
train_ds = CredDataset(train_records)
loader = DataLoader(train_ds, batch_size=8, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridCredModel(transformer_name="distilbert-base-uncased", meta_dim=16).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.MSELoss()

for epoch in range(4):
    model.train()
    for input_ids, attention_mask, meta, score in loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        meta = meta.to(device)
        score = score.to(device)
        pred, gates = model(input_ids, attention_mask, meta)
        loss = criterion(pred, score)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(f"Epoch {epoch} done, loss={loss.item():.4f}")
# After training, save model and tokenizer
torch.save(model.state_dict(), "hybrid_cred_model.pt")
tokenizer.save_pretrained("tokenizer/")


In [ ]:
# deploy_to_hf.py
# Upload model files (weights + tokenizer + small wrapper) to HF Hub
from huggingface_hub import HfApi, Repository, upload_file
import os

HF_USER = "your-hf-username"
REPO_NAME = "hybrid-credibility-model"
LOCAL_FOLDER = "./hf_model"

# ensure files exist: hybrid_cred_model.pt, tokenizer/* etc.
os.makedirs(LOCAL_FOLDER, exist_ok=True)
# copy files into LOCAL_FOLDER (implementation detail omitted)

api = HfApi()
# create repo if not exists (requires HF token via env HF_TOKEN)
api.create_repo(repo_id=f"{HF_USER}/{REPO_NAME}", private=False, exist_ok=True)

# upload files
upload_file(path_or_fileobj="hybrid_cred_model.pt", path_in_repo="hybrid_cred_model.pt", repo_id=f"{HF_USER}/{REPO_NAME}")
for fname in ["tokenizer/vocab.json", "tokenizer/tokenizer.json", "tokenizer/special_tokens_map.json"]:
    if os.path.exists(fname):
        upload_file(path_or_fileobj=fname, path_in_repo=fname, repo_id=f"{HF_USER}/{REPO_NAME}")

print("Uploaded artifacts to Hugging Face Hub.")


For full HF Model Card, include modelcard.md describing intended use, limitations, and evaluation metrics.

Use transformers AutoModelForSequenceClassification if you want Hugging Face-native model class; for custom hybrid, provide a wrapper class in the repo that reconstructs the PyTorch model.

In [ ]:
# app.py
# 3-5 lines of comments for blocks: FastAPI app to serve scoring endpoint to chatbot
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, HttpUrl
from credibility import score_url, score_urls
import uvicorn

app = FastAPI(title="Credibility API")

class SingleURL(BaseModel):
    url: HttpUrl

class BatchURLs(BaseModel):
    urls: list[HttpUrl]

@app.post("/score")
async def score(single: SingleURL):
    # Single synchronous scoring (fast); suitable for Chatbot integration
    result = score_url(single.url)
    if result.get("error"):
        raise HTTPException(status_code=502, detail=result["error"])
    return result

@app.post("/score_batch")
async def score_batch(batch: BatchURLs):
    results = score_urls(batch.urls)
    return {"results": results}

# Run with: uvicorn app:app --host 0.0.0.0 --port 8000
if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000, workers=2)


For high concurrency in production, run with multiple workers behind a load balancer. Use an async HTTP client (httpx.AsyncClient) to parallelize fetches if you expect many concurrent requests — the current module is synchronous to keep behavior simpler for safe resource management.

In [ ]:
# tests/test_credibility.py
# Basic unit tests for the scoring module (use pytest)
import pytest
from credibility import score_url

def test_invalid_url():
    res = score_url("http://nonexistent.invalid.example")
    assert res["score"] == 0 or res["label"] == "Unavailable"

def test_google_com():
    res = score_url("https://www.google.com")
    assert "domain" in res
    assert 0 <= res.get("score", 0) <= 100

# Add more tests for local HTML fixtures to deterministically test heuristics


I

Unit tests: cover input parsing, network failure handling, domain extraction, small HTML fixtures.

Integration tests: run FastAPI endpoints using TestClient and mocked HTTP responses (e.g., responses or httpretty) to avoid network dependency.

Model evaluation: use MSE / MAE for regression or accuracy/F1 for classes. Report ROC AUC if probabilistic. Provide a held-out test set and cross-validation.

Stress testing & performance: run load tests (e.g., locust or k6) against FastAPI, measure p95 latency. Use caching (Redis) for repeated URLs to avoid re-fetch.

II

Metrics: request latency, request success/fail rates, p95 latency, model inference time, cache hit ratio. Export to Prometheus.

Logging: structured logs (JSON) including URL, uid, status_code, score, latency, and errors. Mask PII.

Alerts: high error rate or sudden drop in average score might indicate crawling problems or dataset drift.

III

Validate/whitelist input domains or use a safe browsing list if your service will fetch arbitrary URLs.

Run fetches from an isolated environment (container) with network egress control.

Respect robots.txt if needed, and rate-limit requests per origin.

Limit HTML parse size and timeouts to avoid DoS.